In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import alpaca_trade_api as tradeapi
import numpy as np
import requests
from talib import MACD, RSI, BBANDS
from datetime import datetime, timedelta, date
from IPython.display import display, HTML
%matplotlib inline
import json
import math
from dateutil import parser
from pytz import timezone
import pytz
import sys
import iso8601

In [2]:
api = tradeapi.REST(base_url="https://api.alpaca.markets")
session = requests.session()

In [3]:
symbol = 'CLPS'

In [4]:
day_to_analyze = '2020-10-23'

In [5]:
minutes_data = api.polygon.historic_agg_v2(
    symbol, 
    1, 
    'minute',
    _from = str((datetime.strptime(day_to_analyze, '%Y-%m-%d')-timedelta(days=7)).date()),
    to=str((datetime.strptime(day_to_analyze, '%Y-%m-%d')+timedelta(days=1)).date())).df

In [6]:
minutes_data

,open,high,low,close,volume,vwap
timestamp,,,,,,
2020-10-16 09:11:00-04:00,3.2800,3.2800,3.2800,3.2800,200.0,3.2800
2020-10-16 09:30:00-04:00,3.1300,3.1300,3.1248,3.1300,2403.0,3.1279
2020-10-16 09:31:00-04:00,3.2400,3.2400,3.2400,3.2400,100.0,3.2400
2020-10-16 09:33:00-04:00,3.2400,3.2400,3.1800,3.1800,2177.0,3.1839
2020-10-16 09:41:00-04:00,3.1776,3.1776,3.1000,3.1000,250.0,3.1470
...,...,...,...,...,...,...
2020-10-23 18:47:00-04:00,4.1700,4.1700,4.1700,4.1700,193.0,4.1700
2020-10-23 19:23:00-04:00,4.1700,4.1700,4.1700,4.1700,599.0,4.1700
2020-10-23 19:24:00-04:00,4.1600,4.1600,4.1600,4.1600,1151.0,4.1600


In [7]:
clean_close = minutes_data["close"].dropna().between_time("9:30", "16:00").dropna()

In [8]:
clean_close

timestamp
2020-10-16 09:30:00-04:00    3.1300
2020-10-16 09:31:00-04:00    3.2400
2020-10-16 09:33:00-04:00    3.1800
2020-10-16 09:41:00-04:00    3.1000
2020-10-16 09:42:00-04:00    3.1008
                              ...  
2020-10-23 15:56:00-04:00    4.2500
2020-10-23 15:57:00-04:00    4.2500
2020-10-23 15:58:00-04:00    4.2000
2020-10-23 15:59:00-04:00    4.3400
2020-10-23 16:00:00-04:00    4.3300
Name: close, Length: 1192, dtype: float64

In [9]:
macds = MACD(clean_close)

In [10]:
macds

(timestamp
 2020-10-16 09:30:00-04:00         NaN
 2020-10-16 09:31:00-04:00         NaN
 2020-10-16 09:33:00-04:00         NaN
 2020-10-16 09:41:00-04:00         NaN
 2020-10-16 09:42:00-04:00         NaN
                                ...   
 2020-10-23 15:56:00-04:00   -0.040314
 2020-10-23 15:57:00-04:00   -0.035686
 2020-10-23 15:58:00-04:00   -0.035643
 2020-10-23 15:59:00-04:00   -0.024035
 2020-10-23 16:00:00-04:00   -0.015464
 Length: 1192, dtype: float64,
 timestamp
 2020-10-16 09:30:00-04:00         NaN
 2020-10-16 09:31:00-04:00         NaN
 2020-10-16 09:33:00-04:00         NaN
 2020-10-16 09:41:00-04:00         NaN
 2020-10-16 09:42:00-04:00         NaN
                                ...   
 2020-10-23 15:56:00-04:00   -0.031401
 2020-10-23 15:57:00-04:00   -0.032258
 2020-10-23 15:58:00-04:00   -0.032935
 2020-10-23 15:59:00-04:00   -0.031155
 2020-10-23 16:00:00-04:00   -0.028017
 Length: 1192, dtype: float64,
 timestamp
 2020-10-16 09:30:00-04:00         NaN
 2020-10

In [11]:
macds[0].dropna().to_csv(f"{symbol}-{day_to_analyze}-MACD.csv")
macds[1].dropna().to_csv(f"{symbol}-{day_to_analyze}-MACD-signal.csv")
macds[2].dropna().to_csv(f"{symbol}-{day_to_analyze}-MACD-hist.csv")

In [13]:
macds[0]["2020-10-23 10:06:00":]

timestamp
2020-10-23 10:06:00-04:00    0.259833
2020-10-23 10:07:00-04:00    0.269320
2020-10-23 10:08:00-04:00    0.257729
2020-10-23 10:09:00-04:00    0.232948
2020-10-23 10:10:00-04:00    0.198935
                               ...   
2020-10-23 15:56:00-04:00   -0.040314
2020-10-23 15:57:00-04:00   -0.035686
2020-10-23 15:58:00-04:00   -0.035643
2020-10-23 15:59:00-04:00   -0.024035
2020-10-23 16:00:00-04:00   -0.015464
Length: 343, dtype: float64

In [14]:
macds[1]["2020-10-23 10:06:00":]

timestamp
2020-10-23 10:06:00-04:00    0.225314
2020-10-23 10:07:00-04:00    0.234115
2020-10-23 10:08:00-04:00    0.238838
2020-10-23 10:09:00-04:00    0.237660
2020-10-23 10:10:00-04:00    0.229915
                               ...   
2020-10-23 15:56:00-04:00   -0.031401
2020-10-23 15:57:00-04:00   -0.032258
2020-10-23 15:58:00-04:00   -0.032935
2020-10-23 15:59:00-04:00   -0.031155
2020-10-23 16:00:00-04:00   -0.028017
Length: 343, dtype: float64

In [15]:
macds[2]["2020-10-23 10:06:00":]

timestamp
2020-10-23 10:06:00-04:00    0.034519
2020-10-23 10:07:00-04:00    0.035205
2020-10-23 10:08:00-04:00    0.018891
2020-10-23 10:09:00-04:00   -0.004712
2020-10-23 10:10:00-04:00   -0.030980
                               ...   
2020-10-23 15:56:00-04:00   -0.008913
2020-10-23 15:57:00-04:00   -0.003429
2020-10-23 15:58:00-04:00   -0.002708
2020-10-23 15:59:00-04:00    0.007120
2020-10-23 16:00:00-04:00    0.012553
Length: 343, dtype: float64